In [1]:
!pip install transformers==4.29.0 datasets
!pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.7 MB/s eta 0:00:00
Looking in inde

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
# from datasets import load_dataset
# dataset_train_dev = load_dataset("emily49/hateful_memes_train_dev", split="train+validation")


from datasets import load_dataset
dataset_train_dev = load_dataset("emily49/hateful_memes_train_dev")

  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
feature_extractor

ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [3]:
def process_example(example):
    inputs = feature_extractor(example['image'], return_tensors='pt')
    inputs['labels'] = example['label']
    return inputs

In [ ]:
process_example(dataset_train_dev[0])

{'pixel_values': tensor([[[[-0.3804, -0.0431, -0.0431,  ..., -0.0118, -0.0275, -0.3647],
          [ 0.0353,  0.6471,  0.6549,  ...,  0.6941,  0.6549,  0.0588],
          [ 0.0667,  0.6392,  0.6471,  ...,  0.6784,  0.6706,  0.0431],
          ...,
          [ 0.0431,  0.6000,  0.6549,  ...,  0.7412,  0.7098,  0.0824],
          [ 0.0275,  0.6000,  0.6549,  ...,  0.7098,  0.6941,  0.0902],
          [-0.1686,  0.2235,  0.2863,  ...,  0.3176,  0.3020, -0.1608]],

         [[-0.3490, -0.0118, -0.0118,  ...,  0.0275,  0.0118, -0.3412],
          [ 0.0667,  0.6784,  0.6863,  ...,  0.7333,  0.6941,  0.0824],
          [ 0.0902,  0.6706,  0.6784,  ...,  0.7176,  0.7098,  0.0745],
          ...,
          [-0.1059,  0.4039,  0.4118,  ...,  0.4745,  0.4824, -0.0667],
          [-0.1216,  0.4039,  0.4196,  ...,  0.4588,  0.4667, -0.0588],
          [-0.3176,  0.0353,  0.0588,  ...,  0.0745,  0.0667, -0.3098]],

         [[-0.3569, -0.0196, -0.0196,  ...,  0.0118, -0.0039, -0.3490],
          [ 0

In [4]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in example_batch['image']], return_tensors='pt')

    inputs['labels'] = example_batch['label']
    return inputs

prepared_ds = dataset_train_dev.with_transform(transform)

In [ ]:
prepared_ds['train'][0:2]

{'pixel_values': tensor([[[[-0.3804, -0.0431, -0.0431,  ..., -0.0118, -0.0275, -0.3647],
          [ 0.0353,  0.6471,  0.6549,  ...,  0.6941,  0.6549,  0.0588],
          [ 0.0667,  0.6392,  0.6471,  ...,  0.6784,  0.6706,  0.0431],
          ...,
          [ 0.0431,  0.6000,  0.6549,  ...,  0.7412,  0.7098,  0.0824],
          [ 0.0275,  0.6000,  0.6549,  ...,  0.7098,  0.6941,  0.0902],
          [-0.1686,  0.2235,  0.2863,  ...,  0.3176,  0.3020, -0.1608]],

         [[-0.3490, -0.0118, -0.0118,  ...,  0.0275,  0.0118, -0.3412],
          [ 0.0667,  0.6784,  0.6863,  ...,  0.7333,  0.6941,  0.0824],
          [ 0.0902,  0.6706,  0.6784,  ...,  0.7176,  0.7098,  0.0745],
          ...,
          [-0.1059,  0.4039,  0.4118,  ...,  0.4745,  0.4824, -0.0667],
          [-0.1216,  0.4039,  0.4196,  ...,  0.4588,  0.4667, -0.0588],
          [-0.3176,  0.0353,  0.0588,  ...,  0.0745,  0.0667, -0.3098]],

         [[-0.3569, -0.0196, -0.0196,  ...,  0.0118, -0.0039, -0.3490],
          [ 0

In [5]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [6]:
!pip install evaluate 

import numpy as np
import evaluate

metric = evaluate.load("accuracy")
metric2 = evaluate.load("roc_auc")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from datasets import load_metric

# metric = load_metric("accuracy")
# def compute_metrics(p):
#     return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = metric.compute(predictions=predictions, references=labels)["accuracy"]
    roc = metric2.compute(prediction_scores=predictions, references=labels)["roc_auc"]
    return {"accuracy": acc, "roc_auc": roc}

In [8]:
from transformers import ViTForImageClassification


model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=2
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
cd gdrive/MyDrive/models

/content/gdrive/MyDrive/models


In [ ]:
# if split dataset

split_dataset = prepared_ds.train_test_split(test_size=0.2)

In [10]:
# use wandb for live logging

!pip install wandb

import wandb
wandb.login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


wandb: Currently logged in as: emily49 (emilycs229). Use `wandb login --relogin` to force relogin


True

In [11]:
%env WANDB_PROJECT=hateful_memes_vit

env: WANDB_PROJECT=hateful_memes_vit


In [12]:
run = wandb.init(project="hateful_memes_vit")

In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit",
  # per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  # save_steps=100,
  eval_steps=100,
  save_strategy="epoch",
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='wandb',
  # load_best_model_at_end=True,
)

In [14]:


from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor,
)

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

In [ ]:
wandb.finish()